In [29]:
import mlflow
import json


In [30]:
mlflow.set_tracking_uri("http://91.99.14.73:8080")
mlflow.set_experiment(experiment_name="config-validation")

runs = mlflow.search_runs()
runs.head()


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.indexing_duration,metrics.VAL.SYS..memory_process_mean_mb,metrics.VAL.E2E..MatthewsCorrCoefMetric,metrics.VAL.SYS..cpu_process_min,...,params.components.reranker.init_parameters.calibration_factor,params.metadata.Comment,params.metadata.Reasoning,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.git.commit,tags.LocalEmbedder,tags.Succesful
0,7a404588695e46899981deec7af4b9fb,113580098883527177,RUNNING,mlflow-artifacts:/113580098883527177/7a4045886...,2025-04-28 08:42:04.663000+00:00,NaT,2161.268525,NaN,NaN,NaN,...,None,None,None,/srv/master-thesis-alj95/code/ragnroll_project...,LOCAL,root,from_matrix.rag_hybrid_retriever_0.yaml,5ec5f265dd41e8f579845bb77d3b241f39fea2a6,None,None
1,5ce10fdfaaa749d488f3dd4f6e2d6341,113580098883527177,FINISHED,mlflow-artifacts:/113580098883527177/5ce10fdfa...,2025-04-28 07:18:13.047000+00:00,2025-04-28 08:01:36.013000+00:00,2196.897666,4770.756956,0.301511,0.0,...,None,None,None,/srv/master-thesis-alj95/code/ragnroll_project...,LOCAL,root,from_matrix.rag_hybrid_retriever_0.yaml,5ec5f265dd41e8f579845bb77d3b241f39fea2a6,None,None
2,04cc45743ea54f6daaf03c6af2616f42,113580098883527177,RUNNING,mlflow-artifacts:/113580098883527177/04cc45743...,2025-04-27 15:09:44.001000+00:00,NaT,NaN,NaN,NaN,NaN,...,None,None,None,/srv/master-thesis-alj95/code/ragnroll_project...,LOCAL,root,from_pipeline.cp0_rag_dense_4r_hf_embedder.yaml,be1a7ce3b888a40f5abfedafb17a7db043126dbf,True,None
3,30ce7e43e39a4ec5a3cb138b4e961e78,113580098883527177,FINISHED,mlflow-artifacts:/113580098883527177/30ce7e43e...,2025-04-27 03:06:32.862000+00:00,2025-04-27 11:45:15.568000+00:00,0.021253,NaN,0.000000,0.0,...,None,None,None,/srv/master-thesis-alj95/code/ragnroll_project...,LOCAL,root,from_matrix.rag_dense_4r_iterative_max5_ciri_1...,5d5f4a81030a9102e49636b7f042776935b98b6c,None,None
4,70028b7cd3054ec69b802bd46f7327b2,113580098883527177,FINISHED,mlflow-artifacts:/113580098883527177/70028b7cd...,2025-04-26 18:42:39.803000+00:00,2025-04-27 03:06:32.838000+00:00,0.046402,NaN,0.000000,0.0,...,None,None,None,/srv/master-thesis-alj95/code/ragnroll_project...,LOCAL,root,from_matrix.rag_dense_4r_iterative_max5_ciri_0...,5d5f4a81030a9102e49636b7f042776935b98b6c,None,None


In [31]:
new_run_params = {}
existing_types = []
for run_id in runs["run_id"]:
    new_run_params[run_id] = {}
    print("-" * 100)
    run = runs[runs["run_id"] == run_id].iloc[0]
    for key, value in run.items():
        if key.endswith(".type") and value and value != "LOCAL":
            new_value = "includes." + value.split(".")[-1]
            count_key = "Nr.of." + value.split(".")[-1]
            print(value, "=", True)
            new_run_params[run_id][new_value] = True

            if count_key not in new_run_params[run_id]:
                new_run_params[run_id][count_key] = 1
            else:
                new_run_params[run_id][count_key] += 1

            if new_value not in existing_types:
                existing_types.append(new_value)
existing_types


----------------------------------------------------------------------------------------------------
haystack.components.builders.answer_builder.AnswerBuilder = True
pipelines.components.embedder_adapter.EmbedderAdapter = True
haystack.components.retrievers.in_memory.bm25_retriever.InMemoryBM25Retriever = True
haystack.components.embedders.openai_text_embedder.OpenAITextEmbedder = True
haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever = True
haystack.components.generators.openai.OpenAIGenerator = True
haystack.document_stores.in_memory.document_store.InMemoryDocumentStore = True
haystack.components.builders.prompt_builder.PromptBuilder = True
haystack.document_stores.in_memory.document_store.InMemoryDocumentStore = True
haystack.components.joiners.document_joiner.DocumentJoiner = True
----------------------------------------------------------------------------------------------------
haystack.components.builders.answer_builder.AnswerBuilder = True


['includes.AnswerBuilder',
 'includes.EmbedderAdapter',
 'includes.InMemoryBM25Retriever',
 'includes.OpenAITextEmbedder',
 'includes.InMemoryEmbeddingRetriever',
 'includes.OpenAIGenerator',
 'includes.InMemoryDocumentStore',
 'includes.PromptBuilder',
 'includes.DocumentJoiner',
 'includes.SentenceTransformersTextEmbedder',
 'includes.LostInTheMiddleRanker',
 'includes.OutputAdapter',
 'includes.single',
 'includes.env_var',
 'includes.IterativeEmbedding',
 'includes.TransformersSimilarityRanker']

In [32]:
for run_id, params in new_run_params.items():
    for existing_type in existing_types:
        if existing_type not in params:
            params[existing_type] = False
            params[f"Nr.of.{existing_type.split('.')[-1]}"] = 0
params

{'includes.AnswerBuilder': True,
 'Nr.of.AnswerBuilder': 1,
 'includes.OpenAIGenerator': True,
 'Nr.of.OpenAIGenerator': 1,
 'includes.PromptBuilder': True,
 'Nr.of.PromptBuilder': 1,
 'includes.EmbedderAdapter': False,
 'Nr.of.EmbedderAdapter': 0,
 'includes.InMemoryBM25Retriever': False,
 'Nr.of.InMemoryBM25Retriever': 0,
 'includes.OpenAITextEmbedder': False,
 'Nr.of.OpenAITextEmbedder': 0,
 'includes.InMemoryEmbeddingRetriever': False,
 'Nr.of.InMemoryEmbeddingRetriever': 0,
 'includes.InMemoryDocumentStore': False,
 'Nr.of.InMemoryDocumentStore': 0,
 'includes.DocumentJoiner': False,
 'Nr.of.DocumentJoiner': 0,
 'includes.SentenceTransformersTextEmbedder': False,
 'Nr.of.SentenceTransformersTextEmbedder': 0,
 'includes.LostInTheMiddleRanker': False,
 'Nr.of.LostInTheMiddleRanker': 0,
 'includes.OutputAdapter': False,
 'Nr.of.OutputAdapter': 0,
 'includes.single': False,
 'Nr.of.single': 0,
 'includes.env_var': False,
 'Nr.of.env_var': 0,
 'includes.IterativeEmbedding': False,
 'Nr

In [33]:
hypotheses = []
from uuid import uuid4
for run_id, params in runs.iterrows():
    for key, value in params.items():
        if "hypothesis" in str(key) and value:
            hypotheses.append(value)

hypotheses = list(set(hypotheses))
hypotheses

['Hybrid RAGs have a higher retrieval quality than sparse or dense RAGs.',
 'Most recent LLM models do not need retrieval for configuration validation and local models are sufficient.',
 'Iterative embedding and retrieval for each configuration item can improve the performance of the pipeline.',
 'The top models gemini-2.0 and deepseek gpt-4o handle this task well, how about Coding Models?',
 'Most recent LLM models do not need retrieval for configuration validation and OS models are sufficient.',
 'Most recent LLM models with web search do not need a customized RAG for configuration validation.',
 'Most recent LLM models do not need retrieval for configuration validation.',
 'Dense RAGs have a higher retrieval quality than sparse RAGs.',
 'Sparse RAGs have a higher retrieval quality than dense RAGs.']

In [34]:
hypothesis_dict = {}
for hypothesis in hypotheses:
    hypothesis_dict[hypothesis] = str(uuid4())[:5]

for k, v in hypothesis_dict.items():
    print(v, k)

0e642 Hybrid RAGs have a higher retrieval quality than sparse or dense RAGs.
11ce5 Most recent LLM models do not need retrieval for configuration validation and local models are sufficient.
41ee4 Iterative embedding and retrieval for each configuration item can improve the performance of the pipeline.
ce287 The top models gemini-2.0 and deepseek gpt-4o handle this task well, how about Coding Models?
be30b Most recent LLM models do not need retrieval for configuration validation and OS models are sufficient.
8ff9f Most recent LLM models with web search do not need a customized RAG for configuration validation.
fb8b3 Most recent LLM models do not need retrieval for configuration validation.
efba7 Dense RAGs have a higher retrieval quality than sparse RAGs.
983b9 Sparse RAGs have a higher retrieval quality than dense RAGs.


In [35]:

hypothesis_params = (
    runs
    .loc[:, ["run_id", "params.metadata.hypothesis"]]
    .fillna("No hypothesis")
    .set_index("run_id")
    .assign(value=True)
    .pivot(columns=["params.metadata.hypothesis"])
    .fillna("False")
    )
hypothesis_params.head()


value  \
params.metadata.hypothesis       Dense RAGs have a higher retrieval quality than sparse RAGs.   
run_id                                                                                          
04a7144838064d4cb5d621dc85ec675c                                              False             
04cc45743ea54f6daaf03c6af2616f42                                              False             
30ce7e43e39a4ec5a3cb138b4e961e78                                              False             
414d52e2f6d843b6ac60a2b6221e2a38                                              False             
4a2d3bd7e1124ec786285bf6294b7c3b                                              False             

                                                                                                         \
params.metadata.hypothesis       Hybrid RAGs have a higher retrieval quality than sparse or dense RAGs.   
run_id                                                                                                    
04a7144838064d4cb5d621dc85ec675c                                              False                       
04cc45743ea54f6daaf03c6af2616f42                                              False                       
30ce7e43e39a4ec5a3cb138b4e961e78                                              False                       
414d52e2f6d843b6ac60a2b6221e2a38                                              False                       
4a2d3bd7e1124ec786285bf6294b7c3b                                              False                       

                                                                                                                                             \
params.metadata.hypothesis       Iterative embedding and retrieval for each configuration item can improve the performance of the pipeline.   
run_id                                                                                                                                        
04a7144838064d4cb5d621dc85ec675c                                               True                                                           
04cc45743ea54f6daaf03c6af2616f42                                              False                                                           
30ce7e43e39a4ec5a3cb138b4e961e78                                               True                                                           
414d52e2f6d843b6ac60a2b6221e2a38                                              False                                                           
4a2d3bd7e1124ec786285bf6294b7c3b                                              False                                                           

                                                                                                                                          \
params.metadata.hypothesis       Most recent LLM models do not need retrieval for configuration validation and OS models are sufficient.   
run_id                                                                                                                                     
04a7144838064d4cb5d621dc85ec675c                                              False                                                        
04cc45743ea54f6daaf03c6af2616f42                                              False                                                        
30ce7e43e39a4ec5a3cb138b4e961e78                                              False                                                        
414d52e2f6d843b6ac60a2b6221e2a38                                              False                                                        
4a2d3bd7e1124ec786285bf6294b7c3b                                              False                                                        

                                                                                                                                             \
params.metadata.hypothesis       Most recent 

In [36]:

hypothesis_dict2 = {v:k for k, v in hypothesis_dict.items()}
for run_id, params in hypothesis_params.iterrows():
    for key, value in params.items():
        if value == True:
            print(f"{run_id}: {hypothesis_dict.get(key[1], 'No hypothesis')} = True")
            new_run_params[run_id][hypothesis_dict.get(key[1], 'No hypothesis')] = True
        else:
            # print(f"{run_id} ({value} - {key[1]}): {hypothesis_dict2.get(key[1], 'No hypothesis')}")
            new_run_params[run_id][hypothesis_dict.get(key[1], 'No hypothesis')] = False

with open("hypothesis_dict.json", "w") as f:
    json.dump(hypothesis_dict2, f)

hypothesis_dict2


04a7144838064d4cb5d621dc85ec675c: 41ee4 = True
04cc45743ea54f6daaf03c6af2616f42: No hypothesis = True
30ce7e43e39a4ec5a3cb138b4e961e78: 41ee4 = True
414d52e2f6d843b6ac60a2b6221e2a38: No hypothesis = True
4a2d3bd7e1124ec786285bf6294b7c3b: No hypothesis = True
539bba316edb4c8e9d27bbe48357ba21: No hypothesis = True
5ce10fdfaaa749d488f3dd4f6e2d6341: 0e642 = True
5f7efa6ec96a461dbd85733e9bb811b6: fb8b3 = True
62de284c66464ac8b5451115e15abd49: 8ff9f = True
6747a761f9f34303b322790170d893bd: efba7 = True
685700fe904c456c86cf4db2ed378171: No hypothesis = True
69cb5169bdf2409b9baa85627d217108: ce287 = True
70028b7cd3054ec69b802bd46f7327b2: 41ee4 = True
7875a8010ea44e5ebf4a4c26302c2991: No hypothesis = True
7a404588695e46899981deec7af4b9fb: 0e642 = True
85818a4d847d42a38701d4c20d20f849: No hypothesis = True
9299a5dc28204f5f92482e96325031ec: 11ce5 = True
95bddc79059e45919c6c61e34ca52fcc: No hypothesis = True
97e4782a82d4431bb717e89e5806f886: No hypothesis = True
9cdb34230c43406e98563129ad11a3b7: f

{'0e642': 'Hybrid RAGs have a higher retrieval quality than sparse or dense RAGs.',
 '11ce5': 'Most recent LLM models do not need retrieval for configuration validation and local models are sufficient.',
 '41ee4': 'Iterative embedding and retrieval for each configuration item can improve the performance of the pipeline.',
 'ce287': 'The top models gemini-2.0 and deepseek gpt-4o handle this task well, how about Coding Models?',
 'be30b': 'Most recent LLM models do not need retrieval for configuration validation and OS models are sufficient.',
 '8ff9f': 'Most recent LLM models with web search do not need a customized RAG for configuration validation.',
 'fb8b3': 'Most recent LLM models do not need retrieval for configuration validation.',
 'efba7': 'Dense RAGs have a higher retrieval quality than sparse RAGs.',
 '983b9': 'Sparse RAGs have a higher retrieval quality than dense RAGs.'}

In [40]:
runs["params.val_data_path"].unique()


array(['data/processed/config_val/val/evaluation_data.json', ''],
      dtype=object)

In [37]:
# replace nan and na with empty string
runs = runs.fillna("")

for run_id, params in new_run_params.items():
    run = runs[runs["run_id"] == run_id]

    run_metrics = run.loc[:, run.columns.str.startswith("metrics.")]

    # replace empty strings with 0
    run_metrics = run_metrics.replace("", 0)
    run_metrics = run_metrics.astype(float, errors="ignore")
    print(run_metrics.iloc[0].to_dict())

    run_params = run.loc[:, ~run.columns.str.startswith("metrics.")]

    mlflow.set_experiment(experiment_name="config-validation-3")
    mlflow.start_run(run_name=run_id)
    mlflow.log_params(run_params.iloc[0].to_dict())
    mlflow.log_params(params)
    mlflow.log_metrics(run_metrics.iloc[0].to_dict())
    mlflow.end_run()



{'metrics.indexing_duration': 2161.268524646759, 'metrics.VAL.SYS..memory_process_mean_mb': 0.0, 'metrics.VAL.E2E..MatthewsCorrCoefMetric': 0.0, 'metrics.VAL.SYS..cpu_process_min': 0.0, 'metrics.VAL.SYS..duration_seconds': 0.0, 'metrics.VAL.E2E..F1ScoreMetric': 0.0, 'metrics.VAL.RET.embedding_retriever.MAPAtKMetric': 0.0, 'metrics.VAL.E2E..FalsePositiveRateMetric': 0.0, 'metrics.VAL.RET.bm25_retriever.HaystackContextRelevanceMetric': 0.0, 'metrics.VAL.SYS..memory_process_max_mb': 0.0, 'metrics.VAL.GEN.llm.FormatValidatorMetric': 0.0, 'metrics.VAL.SYS..cpu_system_min': 0.0, 'metrics.VAL.SYS..cpu_process_max': 0.0, 'metrics.VAL.SYS..memory_system_percent_min': 0.0, 'metrics.VAL.E2E..PrecisionMetric': 0.0, 'metrics.VAL.SYS..memory_process_min_mb': 0.0, 'metrics.VAL.E2E..RecallMetric': 0.0, 'metrics.VAL.GEN.llm.AnswerRelevancyMetric': 0.0, 'metrics.VAL.E2E..FalseNegativeRateMetric': 0.0, 'metrics.VAL.SYS..memory_system_percent_mean': 0.0, 'metrics.VAL.E2E..AccuracyMetric': 0.0, 'metrics.VA

C:\Users\Besitzer\AppData\Local\Temp\ipykernel_16300\158029723.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  run_metrics = run_metrics.replace("", 0)


🏃 View run 7a404588695e46899981deec7af4b9fb at: http://91.99.14.73:8080/#/experiments/525945623195643983/runs/35447bab31654822bdb71d02ba846935
🧪 View experiment at: http://91.99.14.73:8080/#/experiments/525945623195643983
{'metrics.indexing_duration': 2196.897666454315, 'metrics.VAL.SYS..memory_process_mean_mb': 4770.756956018518, 'metrics.VAL.E2E..MatthewsCorrCoefMetric': 0.30151134457776363, 'metrics.VAL.SYS..cpu_process_min': 0.0, 'metrics.VAL.SYS..duration_seconds': 396.897874, 'metrics.VAL.E2E..F1ScoreMetric': 0.6666666666666666, 'metrics.VAL.RET.embedding_retriever.MAPAtKMetric': 0.016666666666666666, 'metrics.VAL.E2E..FalsePositiveRateMetric': 0.4, 'metrics.VAL.RET.bm25_retriever.HaystackContextRelevanceMetric': 0.3, 'metrics.VAL.SYS..memory_process_max_mb': 5472.7890625, 'metrics.VAL.GEN.llm.FormatValidatorMetric': 1.0, 'metrics.VAL.SYS..cpu_system_min': 0.0, 'metrics.VAL.SYS..cpu_process_max': 103.9, 'metrics.VAL.SYS..memory_system_percent_min': 31.9, 'metrics.VAL.E2E..Precisi

C:\Users\Besitzer\AppData\Local\Temp\ipykernel_16300\158029723.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  run_metrics = run_metrics.replace("", 0)


🏃 View run 5ce10fdfaaa749d488f3dd4f6e2d6341 at: http://91.99.14.73:8080/#/experiments/525945623195643983/runs/0b85b6ea813b4abda5035150f7494989
🧪 View experiment at: http://91.99.14.73:8080/#/experiments/525945623195643983
{'metrics.indexing_duration': 0.0, 'metrics.VAL.SYS..memory_process_mean_mb': 0.0, 'metrics.VAL.E2E..MatthewsCorrCoefMetric': 0.0, 'metrics.VAL.SYS..cpu_process_min': 0.0, 'metrics.VAL.SYS..duration_seconds': 0.0, 'metrics.VAL.E2E..F1ScoreMetric': 0.0, 'metrics.VAL.RET.embedding_retriever.MAPAtKMetric': 0.0, 'metrics.VAL.E2E..FalsePositiveRateMetric': 0.0, 'metrics.VAL.RET.bm25_retriever.HaystackContextRelevanceMetric': 0.0, 'metrics.VAL.SYS..memory_process_max_mb': 0.0, 'metrics.VAL.GEN.llm.FormatValidatorMetric': 0.0, 'metrics.VAL.SYS..cpu_system_min': 0.0, 'metrics.VAL.SYS..cpu_process_max': 0.0, 'metrics.VAL.SYS..memory_system_percent_min': 0.0, 'metrics.VAL.E2E..PrecisionMetric': 0.0, 'metrics.VAL.SYS..memory_process_min_mb': 0.0, 'metrics.VAL.E2E..RecallMetric':

...

🏃 View run 9cdb34230c43406e98563129ad11a3b7 at: http://91.99.14.73:8080/#/experiments/525945623195643983/runs/a92a9e9107334e6691dd2216c8ce61ee
🧪 View experiment at: http://91.99.14.73:8080/#/experiments/525945623195643983
{'metrics.indexing_duration': 0.0006289482116699219, 'metrics.VAL.SYS..memory_process_mean_mb': 0.0, 'metrics.VAL.E2E..MatthewsCorrCoefMetric': 0.30264057210734246, 'metrics.VAL.SYS..cpu_process_min': 0.0, 'metrics.VAL.SYS..duration_seconds': 4944.289814, 'metrics.VAL.E2E..F1ScoreMetric': 0.6928799149840595, 'metrics.VAL.RET.embedding_retriever.MAPAtKMetric': 0.0, 'metrics.VAL.E2E..FalsePositiveRateMetric': 0.7097625329815304, 'metrics.VAL.RET.bm25_retriever.HaystackContextRelevanceMetric': 0.0, 'metrics.VAL.SYS..memory_process_max_mb': 0.0, 'metrics.VAL.GEN.llm.FormatValidatorMetric': 1.0, 'metrics.VAL.SYS..cpu_system_min': 1.0, 'metrics.VAL.SYS..cpu_process_max': 18.0, 'metrics.VAL.SYS..memory_system_percent_min': 0.0, 'metrics.VAL.E2E..PrecisionMetric': 0.547899159

C:\Users\Besitzer\AppData\Local\Temp\ipykernel_16300\158029723.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  run_metrics = run_metrics.replace("", 0)


🏃 View run d2beb8913af9447488062954606b16d9 at: http://91.99.14.73:8080/#/experiments/525945623195643983/runs/3919a3450f314efb97c86af926246089
🧪 View experiment at: http://91.99.14.73:8080/#/experiments/525945623195643983
{'metrics.indexing_duration': 0.0, 'metrics.VAL.SYS..memory_process_mean_mb': 0.0, 'metrics.VAL.E2E..MatthewsCorrCoefMetric': 0.28435631328396355, 'metrics.VAL.SYS..cpu_process_min': 0.0, 'metrics.VAL.SYS..duration_seconds': 3955.812644, 'metrics.VAL.E2E..F1ScoreMetric': 0.6761565836298933, 'metrics.VAL.RET.embedding_retriever.MAPAtKMetric': 0.0, 'metrics.VAL.E2E..FalsePositiveRateMetric': 0.5593667546174143, 'metrics.VAL.RET.bm25_retriever.HaystackContextRelevanceMetric': 0.0, 'metrics.VAL.SYS..memory_process_max_mb': 0.0, 'metrics.VAL.GEN.llm.FormatValidatorMetric': 0.9986206896551724, 'metrics.VAL.SYS..cpu_system_min': 0.0, 'metrics.VAL.SYS..cpu_process_max': 38.0, 'metrics.VAL.SYS..memory_system_percent_min': 0.0, 'metrics.VAL.E2E..PrecisionMetric': 0.573440643863

C:\Users\Besitzer\AppData\Local\Temp\ipykernel_16300\158029723.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  run_metrics = run_metrics.replace("", 0)


🏃 View run 97e4782a82d4431bb717e89e5806f886 at: http://91.99.14.73:8080/#/experiments/525945623195643983/runs/e8ad0e6c82664fedade47f88e711bf5d
🧪 View experiment at: http://91.99.14.73:8080/#/experiments/525945623195643983
